In [1]:
# Install required libraries here
!pip install --quiet easyocr
!pip install --quiet py7zr

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.9/67.9 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 50.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.3/93.3 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.0/139.0 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.7/413.7 kB 24.6 MB/s eta 0:00:00


In [2]:
# Download dataset from Google Drive
# STRAITS TIMES
#!gdown 1S6AKL6KDmH0yqJM36XK2IrP9pg4vbfCs

# MOTHERSHIP
!gdown 1CtyKcpx2jxP0h9K-IsuIQf8UpsIj7jeO

Downloading...
From (original): https://drive.google.com/uc?id=1CtyKcpx2jxP0h9K-IsuIQf8UpsIj7jeO
From (redirected): https://drive.google.com/uc?id=1CtyKcpx2jxP0h9K-IsuIQf8UpsIj7jeO&confirm=t&uuid=29a7ad81-d956-4222-9104-548d7251e6f6
To: /kaggle/working/with-captions-Mothership.zip
100%|███████████████████████████████████████| 2.76G/2.76G [00:09<00:00, 276MB/s]


In [3]:
import os
import cv2
import easyocr
import pandas as pd
import uuid
from tqdm import tqdm
# Unzip 7zip folder
#import py7zr
# Unzip zip folder
import zipfile

In [4]:
#zipped_path = "/kaggle/working/with-captions-Straits.zip"
zipped_path = "/kaggle/working/with-captions-Mothership.zip"
output_folder = "images"
os.makedirs(output_folder, exist_ok=True)

with zipfile.ZipFile(zipped_path, 'r') as zip_ref:
    zip_ref.extractall(output_folder)

#with py7zr.SevenZipFile(zipped_path, mode='r') as z:
#    # Will extract to "images" folder
#    z.extractall()

In [5]:
reader = easyocr.Reader(['en'], gpu=True) 

In [6]:
folder_path = "/kaggle/working/images"
#output_path = "/kaggle/working/with-captions"
#os.makedirs(output_path, exist_ok=True)

In [7]:
# List to hold captions
caption_data = []

In [8]:
image_files = [f for f in os.listdir(folder_path) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]

In [9]:
for file_name in tqdm(image_files, desc="Processing images"):
    file_path = os.path.join(folder_path, file_name)
    image = cv2.imread(file_path)
    
    if image is None:
        print(f"Could not read {file_name}")
        continue
    
    # Run EasyOCR on the image to detect text
    results = reader.readtext(image)
    
    # If any text is detected, extract the text and record it with the filename
    if results:
        # Join all detected text segments into a single string
        extracted_text = " ".join([text for (_, text, _) in results])
        
        # Append the data to the list
        caption_data.append({
            "FileName": file_name,
            "ExtractedText": extracted_text
        })

Processing images: 100%|██████████| 11192/11192 [1:39:51<00:00,  1.87it/s]


In [10]:
# Save the collected caption data into captions.xlsx
df = pd.DataFrame(caption_data)
excel_path = '/kaggle/working/captions.xlsx'
df.to_excel(excel_path, index=False)

In [11]:
#import shutil

# Define the folder to be zipped and the zip file name.
#folder_to_zip = output_path  # "/kaggle/working/with-captions"
#zip_filename = "with-captions.zip"

# Create a zip archive of the output folder.
#shutil.make_archive("with-captions", "zip", folder_to_zip)